In [1]:
import os

In [2]:
os.chdir("../")

In [4]:
%pwd

'/home/harsh_rana/Projects/Experiments/Deep Learning/Chicken-Disease-Classification-Using-MLOPS-DVC-Pipeline-'

In [13]:
# Entity 

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
# Configuration Manager

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [16]:
#Components data_ingestion

import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
#Pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-02 22:27:12,402:INFO:common:yaml file: config/config.yaml loaded successfully]
[2025-01-02 22:27:12,407:INFO:common:yaml file: params.yaml loaded successfully]
[2025-01-02 22:27:12,409:INFO:common:created directory at: artifacts]
[2025-01-02 22:27:12,412:INFO:common:created directory at: artifacts/data_ingestion]
[2025-01-02 22:27:15,552:INFO:4037744573:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23997306
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "0956ef971811001f1cf77bde0c403c096a66f9d2cdcbcc2b528efa6fcbfa0104"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DA9A:4DE10:7B291A9:804E560:677704B1
Accept-Ranges: bytes
Date: Thu, 02 Jan 2025 21:27:14 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf8230022-FRA
X-Cache: MISS
X-